In [1]:
import numpy as np
from keras.models import Sequential,Model
from keras.layers import Dense, Dropout, Activation, Flatten, Merge
from keras.layers import Convolution1D ,AveragePooling1D
from keras.utils.np_utils import to_categorical

Using Theano backend.


In [2]:
train=open("Downloads/Gun_Point_TRAIN")
test=open("Downloads/Gun_Point_TEST")
train = train.read().split()
test = test.read().split()

In [3]:
train = np.array_split(np.array(train),50)
test = np.array_split(np.array(test),150)
print(np.shape(train))
print(np.shape(test))

(50, 151)
(150, 151)


In [4]:
#get the label of train and test
def Get_label(data):
    label=[]
    for i in range(0,len(data)):
        label.append(float(data[i][0]))
    return label
y_train = Get_label(train)
y_test= Get_label(test)

In [5]:
#get the value of train and test
def Get_value(data):
    value=[]
    for i in range(0,len(data)):
        value.append(data[i][1:151])
    return value
X_train=Get_value(train)
X_test=Get_value(test)

In [6]:
#set the series 
length=150
X_train=np.asarray(X_train)
X_test=np.asarray(X_test)
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)
y_train = np.asarray(y_train)
y_train= y_train.astype(int)
y_test = np.asarray(y_test)
y_test= y_test.astype(int)
y_test=y_test-1
y_train=y_train-1
X_train=X_train.reshape(50,150,1)
X_test=X_test.reshape(150,150,1)

print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)

('X_train shape:', (50, 150, 1))
('X_test shape:', (150, 150, 1))


In [7]:
def process_output(labels):
    return to_categorical(labels, nb_classes=2)
y_train = process_output(y_train)
y_test = process_output(y_test)

In [8]:
print('Building model...')
#first model
model = Sequential()
#1st convolution
model.add(Convolution1D(50,8,input_shape=(150, 1)))
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=2))
#2st convolution
model.add(Convolution1D(100,8))
model.add(Activation('relu'))
model.add(AveragePooling1D(pool_length=2))
model.add(Flatten())
#fulley connected layer
model.add(Dense(500,activation="relu"))
model.add(Dense(2))
#second model
model2 = Sequential()
#1st convolution
model2.add(Convolution1D(50,8,input_shape=(150, 1)))
model2.add(Activation('relu'))
model2.add(AveragePooling1D(pool_length=2))
#2st convolution
model2.add(Convolution1D(100,8))
model2.add(Activation('relu'))
model2.add(AveragePooling1D(pool_length=2))
model2.add(Flatten())
#fulley connected layer
model2.add(Dense(500,activation="relu"))
model2.add(Dense(2))
#merge model
merged = Merge([model, model2], mode='concat')

final_model = Sequential()
final_model.add(merged)
final_model.add(Dense(2, activation='softmax'))

#keras.optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=1e-08)
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])
print(final_model.summary())

Building model...
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution1d_1 (Convolution1D)  (None, 143, 50)       450         convolution1d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 143, 50)       0           convolution1d_1[0][0]            
____________________________________________________________________________________________________
averagepooling1d_1 (AveragePoolin(None, 71, 50)        0           activation_1[0][0]               
____________________________________________________________________________________________________
convolution1d_2 (Convolution1D)  (None, 64, 100)       40100       averagepooling1d_1[0][0]         
_________________________________________________________________________

In [9]:
#input one data is work...
batch_size = 50
nb_epoch = 2
model.fit(X_train, y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          )

Epoch 1/2
50/50 [==============================] - 2s - loss: 8.1560 - acc: 0.5200
Epoch 2/2
50/50 [==============================] - 2s - loss: 7.9390 - acc: 0.4800


In [10]:
#input two data.  GET ERROR
#X_train to CNN1 
#X_test to CNN2
batch_size = 50
nb_epoch = 2
model.fit([X_train,X_test], y_train,
          batch_size=batch_size,
          nb_epoch=nb_epoch,
          )

Exception: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 1 arrays but instead got the following list of 2 arrays: [array([[[-0.64788538],
        [-0.64199156],
        [-0.63818634],
        ..., 
        [-0.64042872],
        [-0.63866574],
        [-0.63865721]],

       [[-0.64442658],
        [-0.64540094],...